In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install opencv-python
!pip install opencv-contrib-python
!apt update && apt install -y libsm6 libxext6
!pip install -q keras

In [0]:
from glob import glob

import cv2
import numpy as np

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dropout, Dense, Flatten
from keras.optimizers import SGD
from keras.losses import categorical_crossentropy
from keras.regularizers import l2
from keras.applications.vgg19 import VGG19

In [0]:
del(model)
del(model_final)

In [0]:
model = VGG19(include_top=False, weights='imagenet', pooling='avg')

In [0]:
for layer in model.layers:
    layer.trainable = False

In [0]:
x = model.output
predictions = Dense(7, activation='softmax')(x)

In [102]:
model_final = Model(input=model.input, output=predictions)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [0]:
from keras.callbacks import ReduceLROnPlateau
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=4, verbose=1)

In [0]:
model_final.compile(loss=categorical_crossentropy,
                  optimizer=SGD(lr=0.001, momentum=0.9, nesterov=True),
                  metrics=['accuracy'])

In [105]:
model_final.fit(np.array(X_train), np.array(y_train),
              batch_size=32,
              epochs=10,
              verbose=1,
              validation_split=0.1,
              shuffle=True)

Train on 29068 samples, validate on 3230 samples
Epoch 1/10
29068/29068 [==============================] - 36s 1ms/step - loss: 1.7122 - acc: 0.3103 - val_loss: 1.6728 - val_acc: 0.3263
Epoch 2/10
 7776/29068 [=======>......................] - ETA: 23s - loss: 1.6566 - acc: 0.333529068/29068 [==============================] - 35s 1ms/step - loss: 1.6454 - acc: 0.3454 - val_loss: 1.6347 - val_acc: 0.3477
Epoch 3/10
20832/29068 [====================>.........] - ETA: 9s - loss: 1.6243 - acc: 0.355629068/29068 [==============================] - 35s 1ms/step - loss: 1.6244 - acc: 0.3560 - val_loss: 1.6151 - val_acc: 0.3539
Epoch 4/10
25760/29068 [=========================>....] - ETA: 3s - loss: 1.6116 - acc: 0.366829068/29068 [==============================] - 35s 1ms/step - loss: 1.6125 - acc: 0.3658 - val_loss: 1.6066 - val_acc: 0.3588
Epoch 5/10
27616/29068 [===========================>..] - ETA: 1s - loss: 1.6028 - acc: 0.368329068/29068 [==============================] - 35s 1ms/step

In [1]:
for layer in model_final.layers[7:]:
  layer.trainable = True

NameError: ignored

In [0]:
model_final.compile(loss=categorical_crossentropy,
                  optimizer=SGD(lr=0.001, momentum=0.9, nesterov=True),
                  metrics=['accuracy'])

In [0]:
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True,
    rotation_range=30,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

train_generator.fit(np.array(X_train))

test_generator = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True)

test_generator.fit(np.array(X_train))

model_final.fit_generator(train_generator.flow(np.array(X_train), np.array(y_train), batch_size=32),
                          steps_per_epoch=len(X_train)/32, 
                          epochs=50, 
                          validation_data=test_generator.flow(np.array(X_test), np.array(y_test), batch_size=32),
                          callbacks=[lr_reducer])

In [78]:
scores = model_final.evaluate(np.array(X_test), np.array(y_test), batch_size=batch_size)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

3589/3589 [==============================] - 3s 885us/step
Loss: 0.9704667871361756
Accuracy: 0.6804123711506282


In [0]:
model.save('./drive/Emotion Recognition/models/model_v7.h5')

In [85]:
!ls

datalab  drive	model_v6.h5


CNN

In [0]:
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy, sparse_categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


MODELPATH = 'drive/Emotion Recognition/models/model_v5.h5'


num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48

In [0]:
import sys
sys.path.insert(0, 'drive/Emotion Recognition/')

In [0]:
data = pd.read_csv('drive/Emotion Recognition/fer2013.csv')

In [16]:
data.tail()

,emotion,pixels,Usage
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest
35886,2,19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...,PrivateTest


In [0]:
def convert_fer2013(data):
    pixels = data['pixels'].tolist()
        
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'), (width, height))
        face = np.stack((face,)*3, -1)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    
    
    emotions = pd.get_dummies(data['emotion']).as_matrix()
    
    return faces, emotions

In [0]:
def normalize(images):
    new_images = []
    for img in images:
        img = img / 255.0
        new_images.append(img)
    return new_images

In [0]:
X, y = convert_fer2013(data)

In [20]:
print(X.shape)

(35887, 48, 48, 3)


In [0]:
X = normalize(X)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=32)

In [0]:
del(model_final)

In [0]:
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

In [51]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_41 (Conv2D)           (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization_36 (Batc (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_36 (Dropout)         (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_37 (Batc (None, 23, 23, 128)       512       
__________

In [0]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7, decay=0.0, amsgrad=False),
              metrics=['accuracy'])

In [0]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)
tensorboard = TensorBoard(log_dir='./logs') 
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(MODELPATH, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [54]:
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          shuffle=True,
          callbacks=[reduce_lr, tensorboard, early_stopper, checkpointer])

Train on 29068 samples, validate on 3589 samples
Epoch 1/100
29068/29068 [==============================] - 69s 2ms/step - loss: 2.0089 - acc: 0.2149 - val_loss: 1.8314 - val_acc: 0.2455

Epoch 00001: val_loss improved from inf to 1.83139, saving model to drive/Emotion Recognition/models/model_v5.h5
Epoch 2/100
 5312/29068 [====>.........................] - ETA: 54s - loss: 1.8511 - acc: 0.234629068/29068 [==============================] - 68s 2ms/step - loss: 1.8336 - acc: 0.2447 - val_loss: 1.8686 - val_acc: 0.2455

Epoch 00002: val_loss did not improve
Epoch 3/100
16832/29068 [================>.............] - ETA: 27s - loss: 1.8172 - acc: 0.244229068/29068 [==============================] - 68s 2ms/step - loss: 1.8135 - acc: 0.2490 - val_loss: 1.8457 - val_acc: 0.2455

Epoch 00003: val_loss did not improve
Epoch 4/100
20480/29068 [====================>.........] - ETA: 19s - loss: 1.7684 - acc: 0.280029068/29068 [==============================] - 68s 2ms/step - loss: 1.7587 - acc:

20032/29068 [===================>..........] - ETA: 20s - loss: 0.9183 - acc: 0.668529068/29068 [==============================] - 68s 2ms/step - loss: 0.9199 - acc: 0.6693 - val_loss: 1.0598 - val_acc: 0.6152

Epoch 00028: val_loss improved from 1.07774 to 1.05978, saving model to drive/Emotion Recognition/models/model_v5.h5
Epoch 29/100
16384/29068 [===============>..............] - ETA: 28s - loss: 0.9085 - acc: 0.670329068/29068 [==============================] - 68s 2ms/step - loss: 0.9094 - acc: 0.6704 - val_loss: 1.1261 - val_acc: 0.5921

Epoch 00029: val_loss did not improve
Epoch 30/100
20288/29068 [===================>..........] - ETA: 19s - loss: 0.8831 - acc: 0.685129068/29068 [==============================] - 68s 2ms/step - loss: 0.8923 - acc: 0.6816 - val_loss: 1.0810 - val_acc: 0.6147

Epoch 00030: val_loss did not improve
Epoch 31/100
21568/29068 [=====================>........] - ETA: 17s - loss: 0.8714 - acc: 0.686929068/29068 [==============================] - 68s 

In [56]:
scores = model.evaluate(np.array(X_test), np.array(y_test), batch_size=batch_size)

3589/3589 [==============================] - 2s 686us/step


In [57]:
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

Loss: 1.0770609875215598
Accuracy: 0.6411256617608261


In [0]:
model.save(MODELPATH)